# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import cv2
import os
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from hmmlearn import hmm
from hmmlearn.hmm import GaussianHMM
import joblib
import re
from scipy.fftpack import dct

# Extracting GLCM features

In [2]:
# Function to compute GLCM features for six properties across four directions
def compute_glcm_features_4_directions(window, distances=[1], levels=256):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Ensure the window pixel intensity is in the range [0, levels-1]
    window = np.clip(window, 0, levels - 1).astype(np.uint8)
    
    # Define angles for the four directions: 0°, 45°, 90°, 135°
    angles = [0, np.pi/2]
    
    # Compute GLCM
    glcm = graycomatrix(window, distances=distances, angles=angles, levels=levels, symmetric=True, normed=True)
    
    # Extract GLCM properties for six features
    features = {}
    features['contrast'] = np.mean(graycoprops(glcm, 'contrast'))
    features['dissimilarity'] = np.mean(graycoprops(glcm, 'dissimilarity'))
    features['homogeneity'] = np.mean(graycoprops(glcm, 'homogeneity'))
    features['ASM'] = np.mean(graycoprops(glcm, 'ASM'))  # Angular Second Moment (Energy)
    features['energy'] = np.mean(np.sqrt(graycoprops(glcm, 'ASM')))  # Energy is sqrt(ASM)
    features['correlation'] = np.mean(graycoprops(glcm, 'correlation'))
    
    return [features['contrast'], features['dissimilarity'], features['homogeneity'], 
            features['ASM'], features['energy'], features['correlation']]


# Extracting DCT features

In [3]:
def compute_dct_features(window):
    # Convert to grayscale if the window is not already
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    
    # Flatten the window to 1D array for DCT computation
    window_flat = window.flatten()
    
    # Compute DCT and take the first N coefficients (e.g., 198)
    dct_features = dct(window_flat, norm='ortho')[:50]
    
    return dct_features

# Combned both GLCM and DCT features

In [4]:
def compute_combined_features(window, dct_features=50):
    # Compute DCT features
    dct_feats = compute_dct_features(window)
    # Compute GLCM features
    glcm_feature = compute_glcm_features_4_directions(window)
    # Combine DCT and GLCM features
    combined_features = np.hstack([dct_feats, glcm_feature])
    return combined_features

# Load Data

In [5]:
labels_df = pd.read_excel(r'//home//jaykishor_c//ml//gt_WIndow .xlsx')  

image_folder = r'//home//jaykishor_c//ml//color_window_double1//color_window_double1'  

In [7]:
# len(combined_features)

# Combined features (GLCM and DCT) sequences for each character across all images

In [9]:
combined_features_glcm_and_dct = {}

window_width = 30   # Width of each sliding window in pixels
step_size = 10     # Step size of the sliding window in pixels

# Process each image (word) in the dataset
for index, row in labels_df.iterrows():
    image_name = row['image name']          # image name of respective gt text
    character_sequence = row['gt']          # gt text
    
    # Load the corresponding image
    image_path = os.path.join(image_folder, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image {image_name} could not be loaded.")
        continue

    image_width = image.shape[1]  # Get image width
    
    # Calculate the width of each character region based on the sequence length
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
    
    # Loop through each character in the sequence and collect its GLCM features
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to the current character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = image[:, region_start:region_end]  # Assume height is all rows
        
        # Split the character region into windows to capture GLCM features
        num_windows = (character_width - window_width) // step_size + 1
        char_combined_sequence = []
        
        for j in range(num_windows):
            # Calculate the start and end of the window within the character region
            window_start = region_start + j * step_size
            window_end = window_start + window_width
            
            # Extract the window
            window = image[:, window_start:window_end]
            
            # Compute GLCM features for this window across four directions
            # glcm_features = compute_glcm_features_4_directions(window)
            # dct_features = compute_dct_features(window)
            # combined_features = np.hstack([glcm_features, dct_features])
            combined_features = compute_combined_features(window)
            # combine_features = combine_glcm_dct_features(dct_features, glcm_features)
            char_combined_sequence.append(combined_features)

        
        # Append this character's GLCM features to the global dictionary
        if char not in combined_features_glcm_and_dct:
            combined_features_glcm_and_dct[char] = []
        combined_features_glcm_and_dct[char].append(char_combined_sequence)

In [10]:
len(char_combined_sequence)

14

# Printing GLCM and DCT features sequences for each character

In [11]:
# Print the number of GLCM feature sequences for each character
for char, sequences in combined_features_glcm_and_dct.items():
    print(f"Character '{char}'  has {len(sequences)}    sequences of GLCM features.")


Character 'െ'  has 618    sequences of GLCM features.
Character 'ത'  has 535    sequences of GLCM features.
Character 'അ'  has 103    sequences of GLCM features.
Character '്'  has 1202    sequences of GLCM features.
Character 'ാ'  has 993    sequences of GLCM features.
Character 'ി'  has 729    sequences of GLCM features.
Character 'പ'  has 339    sequences of GLCM features.
Character 'റ'  has 432    sequences of GLCM features.
Character 'ര'  has 681    sequences of GLCM features.
Character 'ു'  has 455    sequences of GLCM features.
Character 'ള'  has 78    sequences of GLCM features.
Character 'ക'  has 473    sequences of GLCM features.
Character 'ഷ'  has 105    sequences of GLCM features.
Character 'മ'  has 361    sequences of GLCM features.
Character 'ആ'  has 28    sequences of GLCM features.
Character 'ഠ'  has 61    sequences of GLCM features.
Character 'ൽ'  has 40    sequences of GLCM features.
Character 'ർ'  has 62    sequences of GLCM features.
Character 'ഇ'  has 30    sequenc

# Save the GLCM features to a file for analysis


In [12]:
output_path =r'//home//jaykishor_c//ml//Output2//character_glcm_and_dct_features_final.xlsx'

features_df = pd.DataFrame({
    'Character': [char for char, sequences in combined_features_glcm_and_dct.items() for _ in sequences],
    'GLCM Features': [seq for sequences in combined_features_glcm_and_dct.values() for seq in sequences]
})

features_df.to_excel(output_path, index=False)
print(f"GLCM features saved to {output_path}")

GLCM features saved to //home//jaykishor_c//ml//Output2//character_glcm_and_dct_features_final.xlsx


In [13]:
len(char_combined_sequence)

14

# Storing Character HMMs

In [14]:
character_hmms = {}
num_states = 4

# Example: 'character_glcm_sequences' contains the GLCM feature sequences for each character
for char, sequences in combined_features_glcm_and_dct.items():
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    print(len(sequences))
    if len(sequences) == 0:
        print(f"Warning: No valid sequences for character {char}. Skipping this character.")
        continue  # Skip to the next character if no valid sequences
    
    if len(sequences) >= 15:
        # Prepare training data for the HMM
        X = np.vstack(sequences)  # Stack the sequences into a single array
        lengths = [len(seq) for seq in sequences]  # Length of each sequence
        print(lengths)
        # Initialize HMM for this character
        model = hmm.GaussianHMM(n_components=num_states, covariance_type="diag", n_iter=1000, init_params='', verbose=True)


        # Use the custom model
        model.fit(X, lengths)

        # Store the trained model
        character_hmms[char] = model
        print(f"Model trained for character: {char}")

618
[30, 14, 30, 30, 30, 14, 14, 14, 14, 14, 30, 14, 14, 30, 14, 30, 14, 30, 14, 14, 14, 14, 14, 30, 14, 14, 14, 30, 14, 14, 30, 30, 30, 14, 6, 8, 8, 14, 14, 14, 30, 6, 14, 30, 14, 30, 14, 14, 30, 14, 14, 30, 30, 14, 14, 6, 14, 14, 30, 14, 14, 14, 14, 14, 14, 14, 30, 14, 8, 8, 8, 8, 3, 4, 8, 8, 8, 8, 6, 14, 4, 8, 8, 8, 8, 8, 8, 8, 14, 6, 14, 6, 8, 8, 4, 2, 4, 4, 3, 6, 14, 14, 4, 4, 14, 14, 6, 14, 14, 14, 2, 2, 8, 8, 14, 14, 8, 8, 8, 6, 6, 4, 4, 8, 8, 14, 14, 8, 8, 14, 8, 6, 8, 14, 4, 14, 8, 8, 14, 8, 8, 14, 30, 8, 4, 8, 8, 14, 14, 4, 4, 14, 6, 14, 14, 8, 8, 14, 14, 8, 14, 14, 8, 8, 14, 14, 8, 14, 4, 8, 8, 6, 6, 14, 14, 8, 8, 14, 4, 4, 2, 2, 3, 2, 2, 2, 4, 2, 3, 4, 4, 6, 4, 3, 1, 2, 2, 4, 2, 4, 4, 2, 1, 3, 3, 4, 3, 1, 2, 1, 4, 3, 4, 2, 4, 4, 4, 3, 1, 2, 2, 2, 2, 2, 2, 3, 3, 2, 30, 1, 2, 2, 1, 2, 2, 4, 2, 2, 3, 3, 3, 4, 3, 2, 2, 2, 3, 3, 3, 2, 3, 1, 2, 4, 2, 3, 2, 4, 4, 4, 4, 3, 1, 6, 4, 4, 3, 2, 14, 30, 14, 8, 14, 14, 30, 14, 30, 14, 14, 30, 14, 14, 14, 14, 30, 14, 14, 30, 14, 14, 14, 6

         1 -2100229.60589664             +nan
         2 -2043946.39462183  +56283.21127481
         3 -2016519.93395459  +27426.46066724
         4 -2001806.81812657  +14713.11582802
         5 -1997701.78805688   +4105.03006969
         6 -1996877.49976359    +824.28829329
         7 -1996613.63322689    +263.86653670
         8 -1996427.28541274    +186.34781415
         9 -1996273.49120909    +153.79420365
        10 -1996177.40840467     +96.08280442
        11 -1996111.79635517     +65.61204951
        12 -1996065.26750924     +46.52884593
        13 -1996039.13190789     +26.13560135
        14 -1996024.44838520     +14.68352268
        15 -1996015.88040158      +8.56798362
        16 -1996010.53025340      +5.35014819
        17 -1996006.96652900      +3.56372440
        18 -1996004.68133297      +2.28519603
        19 -1996003.31098858      +1.37034439
        20 -1996002.51125299      +0.79973559
        21 -1996002.04804394      +0.46320905
        22 -1996001.77967488      

Model trained for character: െ
525
[14, 8, 8, 14, 6, 6, 6, 6, 8, 14, 30, 14, 30, 8, 14, 30, 30, 14, 8, 8, 14, 6, 8, 14, 8, 8, 14, 14, 30, 8, 8, 8, 8, 14, 8, 14, 8, 8, 8, 6, 4, 6, 6, 14, 8, 8, 6, 8, 4, 4, 3, 3, 6, 8, 8, 8, 8, 14, 14, 6, 8, 8, 14, 14, 14, 4, 8, 6, 6, 8, 6, 14, 8, 4, 4, 8, 8, 8, 6, 4, 4, 4, 6, 6, 6, 6, 8, 8, 8, 14, 6, 8, 14, 14, 4, 6, 14, 14, 6, 8, 14, 6, 6, 8, 6, 8, 2, 4, 14, 30, 14, 8, 6, 3, 3, 4, 8, 6, 6, 4, 14, 14, 8, 6, 6, 8, 14, 8, 14, 6, 2, 4, 6, 6, 8, 14, 14, 6, 8, 6, 8, 4, 4, 4, 2, 2, 1, 1, 2, 3, 2, 1, 3, 3, 2, 4, 2, 3, 1, 1, 3, 2, 2, 3, 2, 2, 4, 3, 2, 2, 2, 4, 1, 2, 2, 1, 3, 2, 4, 3, 3, 2, 2, 4, 3, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 3, 3, 14, 1, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 1, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 2, 2, 2, 1, 1, 2, 2, 3, 2, 2, 3, 2, 3, 2, 3, 2, 6, 3, 3, 2, 2, 2, 4, 8, 4, 3, 2, 3, 3, 3, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 3, 2, 1, 2, 1, 1, 6, 4, 3, 4, 2, 2, 3, 6, 6, 14, 8, 14, 8, 6, 6, 6, 14, 14, 14, 6, 6

         4 -993030.33061590   +5923.13768377
         5 -991925.72427532   +1104.60634057
         6 -991402.86846542    +522.85580991
         7 -990283.87634806   +1118.99211735
         8 -988057.06926462   +2226.80708344
         9 -985903.53132100   +2153.53794362
        10 -984872.01077573   +1031.52054528
        11 -984226.39356856    +645.61720717
        12 -983876.52133404    +349.87223451
        13 -983700.89415766    +175.62717638
        14 -983601.26252816     +99.63162951
        15 -983539.55188359     +61.71064456
        16 -983514.41516596     +25.13671763
        17 -983501.55042672     +12.86473924
        18 -983493.07884405      +8.47158267
        19 -983485.95303341      +7.12581064
        20 -983479.00152627      +6.95150714
        21 -983471.33020857      +7.67131770
        22 -983466.76658486      +4.56362371
        23 -983464.14450196      +2.62208290
        24 -983462.64879371      +1.49570825
        25 -983461.82377352      +0.82502019
        26

Model trained for character: ത
102
[30, 30, 30, 4, 30, 30, 14, 8, 30, 30, 14, 14, 14, 4, 4, 3, 2, 30, 14, 14, 30, 14, 14, 30, 14, 14, 30, 14, 14, 4, 14, 14, 14, 14, 14, 14, 6, 14, 8, 4, 30, 14, 14, 30, 14, 8, 14, 14, 8, 14, 14, 8, 30, 30, 8, 14, 6, 14, 4, 14, 6, 14, 8, 14, 14, 14, 14, 14, 14, 14, 14, 4, 6, 14, 14, 4, 4, 14, 30, 14, 14, 6, 14, 6, 14, 14, 14, 8, 8, 8, 14, 8, 8, 14, 8, 8, 30, 6, 4, 6, 6, 14]
Model trained for character: അ
1151
[8, 6, 6, 8, 8, 8, 6, 6, 4, 6, 4, 8, 4, 6, 8, 8, 6, 6, 3, 4, 8, 8, 8, 6, 4, 6, 6, 8, 6, 6, 8, 6, 8, 6, 6, 6, 6, 8, 8, 3, 3, 8, 3, 3, 6, 4, 4, 8, 6, 8, 8, 8, 8, 8, 8, 3, 3, 2, 2, 3, 3, 2, 2, 2, 3, 3, 4, 4, 2, 2, 6, 4, 6, 3, 4, 4, 4, 6, 3, 2, 2, 2, 6, 3, 3, 4, 4, 6, 4, 6, 6, 3, 3, 3, 3, 6, 6, 4, 4, 4, 4, 4, 6, 3, 3, 3, 2, 2, 3, 3, 4, 6, 4, 4, 6, 6, 4, 4, 4, 2, 2, 6, 6, 3, 6, 6, 8, 4, 3, 3, 6, 6, 3, 3, 3, 2, 2, 6, 6, 6, 8, 8, 6, 4, 4, 4, 4, 3, 6, 8, 4, 2, 2, 2, 2, 6, 4, 4, 6, 4, 6, 4, 4, 3, 3, 4, 6, 4, 6, 2, 2, 2, 2, 4, 4, 3, 3, 6, 4, 4, 4, 6, 4, 4, 4,

...

         1 -187941.14532805             +nan
         2 -183928.06097129   +4013.08435677
         3 -182825.96827722   +1102.09269407
         4 -182126.80382680    +699.16445042
         5 -181984.46332105    +142.34050574
         6 -181876.65536500    +107.80795605
         7 -181793.94016475     +82.71520026
         8 -181730.54737591     +63.39278884
         9 -181712.11808976     +18.42928614
        10 -181701.77730711     +10.34078265
        11 -181690.20130363     +11.57600349
        12 -181688.02320366      +2.17809996
        13 -181682.12111234      +5.90209132
        14 -181675.66050491      +6.46060743
        15 -181671.76299103      +3.89751388
        16 -181663.40789806      +8.35509296
        17 -181649.08986454     +14.31803352
        18 -181636.96911122     +12.12075332
        19 -181628.67682494      +8.29228628
        20 -181603.78214996     +24.89467498
        21 -181596.30972189      +7.47242808
        22 -181593.90012768      +2.40959421
        23

Model trained for character: ൈ
23
[6, 8, 6, 14, 4, 4, 14, 30, 8, 4, 6, 8, 14, 14, 14, 4, 2, 3, 6, 2, 3, 2, 2]
Model trained for character: ഥ
26
[4, 6, 14, 6, 30, 14, 14, 8, 6, 8, 6, 8, 14, 6, 6, 14, 3, 6, 6, 14, 6, 14, 4, 8, 8, 14]
Model trained for character: ൾ
16
[8, 4, 6, 6, 3, 2, 3, 6, 30, 2, 3, 2, 6, 4, 4, 2]
Model trained for character: ഘ
62
[3, 8, 3, 6, 3, 6, 3, 8, 6, 14, 8, 2, 8, 8, 8, 1, 2, 3, 1, 1, 1, 2, 3, 2, 2, 3, 3, 3, 3, 1, 3, 1, 2, 2, 14, 14, 14, 8, 4, 4, 4, 6, 6, 8, 4, 8, 4, 4, 3, 8, 4, 8, 8, 6, 8, 8, 6, 6, 4, 8, 4, 6]


        20  -94312.23931879      +0.15423417
        21  -94311.31718638      +0.92213241
        22  -94303.04618564      +8.27100074
        23  -94294.39321038      +8.65297525
        24  -94294.39146581      +0.00174458
         1 -106316.47777319             +nan
         2 -102994.90026130   +3321.57751189
         3 -102636.86912307    +358.03113822
         4 -102623.90092325     +12.96819982
         5 -102623.12008831      +0.78083495
         6 -102621.52594663      +1.59414168
         7 -102621.52898655      -0.00303993
Model is not converging.  Current: -102621.52898655119 is not greater than -102621.52594662542. Delta is -0.0030399257666431367
         1  -35779.54320628             +nan
         2  -34692.02719070   +1087.51601559
         3  -34221.07755095    +470.94963975
         4  -34185.73225070     +35.34530025
         5  -34173.74117660     +11.99107409
         6  -34151.43129742     +22.30987919
         7  -34151.43129665      +0.00000077
         1  -4499

Model trained for character: ൃ
8
12
3
54
[14, 14, 14, 14, 14, 14, 3, 3, 4, 6, 3, 2, 4, 3, 4, 6, 4, 4, 4, 6, 4, 4, 3, 2, 3, 4, 6, 3, 6, 2, 4, 3, 6, 3, 3, 2, 4, 6, 4, 6, 2, 6, 4, 6, 2, 6, 6, 14, 14, 14, 14, 14, 14, 14]
Model trained for character: ൊ
5
22
[6, 8, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 6, 6, 6, 6, 6, 8, 14, 14, 14]
Model trained for character: ഞ
1
26
[8, 14, 8, 4, 4, 3, 1, 3, 4, 6, 4, 6, 6, 6, 6, 8, 6, 6, 6, 6, 6, 8, 6, 6, 4, 6]
Model trained for character: ബ
4
25
[2, 2, 4, 2, 2, 4, 4, 4, 3, 3, 2, 3, 2, 3, 3, 1, 1, 1, 1, 1, 4, 4, 2, 2, 2]
Model trained for character: ഃ
11
3
11
1
2
3
2
1
4


         4  -43847.95687326     +29.70309901
         5  -43809.42389012     +38.53298314
         6  -43794.03801501     +15.38587511
         7  -43778.44004917     +15.59796585
         8  -43778.41103903      +0.02901014
         9  -43778.40498327      +0.00605576
         1  -18024.21101202             +nan
         2  -17200.48334301    +823.72766901
         3  -17097.38123503    +103.10210798
         4  -17093.95789462      +3.42334041
         5  -17093.95789005      +0.00000456


# View the transition matrix after training

In [15]:
print(model.transmat_)  
row_sums = model.transmat_.sum(axis=1)
print(row_sums)

[[7.77777778e-001 3.88732678e-034 2.22222222e-001 7.44031374e-315]
 [2.23813875e-030 1.00000000e+000 2.74791074e-044 0.00000000e+000]
 [1.63716042e-053 2.64584386e-056 1.00000000e+000 0.00000000e+000]
 [0.00000000e+000 0.00000000e+000 0.00000000e+000 1.00000000e+000]]
[1. 1. 1. 1.]


# Save the trained HMM model to a file 

In [16]:
from joblib import dump

# Assuming your models are in a dictionary called `character_hmms`
# e.g., `character_hmms = {'െ': trained_model_1, 'ത': trained_model_2, ... }

def sanitize_filename(char):
    # Replace invalid filename characters with an underscore or remove them
    return re.sub(r'[<>:"/\\|?*]', '_', char)

# Ensure the directory for the models exists
model_directory = '//home//jaykishor_c//ml//Model_combined'
os.makedirs(model_directory, exist_ok=True)

for char, model in character_hmms.items():
    # Sanitize character to create a valid filename
    sanitized_char = sanitize_filename(char)
    print(char, sanitized_char)
    # Save the trained HMM model to a file with the sanitized name in the 'model' directory
    model_path = os.path.join(model_directory, f"{sanitized_char}_hmm.pkl")
    dump(model, model_path)


െ െ
ത ത
അ അ
് ്
ാ ാ
ി ി
പ പ
റ റ
ര ര
ു ു
ള ള
ക ക
ഷ ഷ
മ മ
ആ ആ
ഠ ഠ
ൽ ൽ
ർ ർ
ഇ ഇ
ങ ങ
ന ന
ജ ജ
ട ട
വ വ
ഹ ഹ
ം ം
യ യ
" _
ധ ധ
ച ച
എ എ
ല ല
ഴ ഴ
ദ ദ
ഗ ഗ
ഉ ഉ
ണ ണ
ൂ ൂ
ൻ ൻ
സ സ
ീ ീ
ഭ ഭ
ഖ ഖ
ശ ശ
ൈ ൈ
ഥ ഥ
ൾ ൾ
ഘ ഘ
ൃ ൃ
ൊ ൊ
ഞ ഞ
ബ ബ
ഃ ഃ


In [17]:
print(X.shape)  

(62, 56)


# Testing shell 1

In [38]:
import joblib
from hmmlearn import hmm
import cv2
import numpy as np
import pandas as pd
 
 
# Function to compute the actual sequence based on window mapping
# Function to extract the actual sequence for a specific image from the DataFrame
def get_actual_sequence_from_df(image_name, line_image, gt_df, window_width=30, step_size=10):
    # Find the corresponding row in the DataFrame
    row = gt_df[gt_df['image name'] == image_name]
    if row.empty:
        raise ValueError(f"Image name '{image_name}' not found in the gts DataFrame.")
    # Extract the ground truth character sequence
    character_sequence = row['gt'].values[0]  # Adjust column name if necessary
    # Compute the actual sequence based on ground truth and image dimensions
    actual_sequence = []
    image_width = line_image.shape[1]
    num_characters = len(character_sequence)
    character_width = image_width // num_characters
 
    for i, char in enumerate(character_sequence):
        # Define the region corresponding to this character
        region_start = i * character_width
        region_end = region_start + character_width
        character_region = line_image[:, region_start:region_end]
 
        # Divide the character region into windows
        num_windows = (character_width - window_width) // step_size + 1
        for _ in range(num_windows):
            actual_sequence.append(char)  # Map each window to the current character
    return actual_sequence
# Function to predict the sequence based on HMM models
def predict_line_sequence(line_image, window_width=30, step_size=10):
    predictions = []
    image_width = line_image.shape[1]
    num_windows = (image_width - window_width) // step_size + 1
 
    for i in range(num_windows):
        window_start = i * step_size
        window_end = window_start + window_width
        window = line_image[:, window_start:window_end]
        test_features = compute_combined_features(window)
 
        test_features = test_features.reshape(1, -1) 

 
        # Calculate likelihoods for each character model
        best_char = None
        best_score = float('-inf')
 
        for char, model in character_hmms.items():
            try:
                score = model.score(test_features )
                if score > best_score:
                    best_score = score
                    best_char = char
            except:
                pass  # Ignore errors for invalid model scoring
 
        if best_char is not None:
            predictions.append(best_char)
 
    return ''.join(predictions)
 
# Test image details
# Load the gts DataFrame
gt_file = r"//home//jaykishor_c//ml//gt_WIndow .xlsx"
gt_df = pd.read_excel(gt_file)

# Test image details
test_image_path = r"//home//jaykishor_c//ml//color_window_double1//color_window_double1//MaI14_007_3.jpg_window_2.jpg"
test_image_name = test_image_path.split("/")[-1]  # Correctly extracts the image name by splitting on '/'

# print("test image name : ", test_image_name)
# print("Data frame image name : ", gt_df['image name'])
 
# Load the test image
test_image = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
 
# Extract the actual sequence from the DataFrame
actual_sequence = get_actual_sequence_from_df(test_image_name, test_image, gt_df)
 
# Predict the sequence using the HMM models
predicted_sequence = predict_line_sequence(test_image)
 
# Print the sequences for comparison
print(f"Actual sequence: {''.join(actual_sequence)}")
print(f"Predicted sequence: {predicted_sequence}")

Actual sequence: വവവവവവവവൃൃൃൃൃൃൃൃഥഥഥഥഥഥഥഥ
Predicted sequence: ദസിവദദമദ്്ൂസസളളസൈൈഭഭർർർ്ർസസർമസ


# Extract the character names from the character_hmms dictionary

In [20]:
char_names = list(character_hmms.keys())

# Print the character names
print("Character names:", char_names)

Character names: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ഖ', 'ശ', 'ൈ', 'ഥ', 'ൾ', 'ഘ', 'ൃ', 'ൊ', 'ഞ', 'ബ', 'ഃ']


In [21]:
len(char_names)

53

In [22]:
ground_truth =char_names

In [23]:
print("Ground truth list:", ground_truth)

Ground truth list: ['െ', 'ത', 'അ', '്', 'ാ', 'ി', 'പ', 'റ', 'ര', 'ു', 'ള', 'ക', 'ഷ', 'മ', 'ആ', 'ഠ', 'ൽ', 'ർ', 'ഇ', 'ങ', 'ന', 'ജ', 'ട', 'വ', 'ഹ', 'ം', 'യ', '"', 'ധ', 'ച', 'എ', 'ല', 'ഴ', 'ദ', 'ഗ', 'ഉ', 'ണ', 'ൂ', 'ൻ', 'സ', 'ീ', 'ഭ', 'ഖ', 'ശ', 'ൈ', 'ഥ', 'ൾ', 'ഘ', 'ൃ', 'ൊ', 'ഞ', 'ബ', 'ഃ']


# Converting 'character to state' and 'state to character'

In [24]:
unique_chars = sorted(set("".join(ground_truth)))
char_to_state = {char: idx for idx, char in enumerate(unique_chars)}
state_to_char = {idx: char for char, idx in char_to_state.items()}

# Testing shel 2

In [45]:
import os
import cv2
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
 
# Function to calculate BLEU score
def calculate_bleu_score(actual_sequence, predicted_sequence):
    reference = [list(actual_sequence)]  # Wrap in another list for multiple references
    hypothesis = list(predicted_sequence)
    bleu_score = sentence_bleu(reference, hypothesis)
    return bleu_score
 
# Function to calculate accuracy
def calculate_accuracy(actual_sequence, predicted_sequence):
    # Ensure the sequences are the same length for comparison
    min_length = min(len(actual_sequence), len(predicted_sequence))
    if min_length == 0:
        return 0  # Return 0 if either sequence is empty to avoid division by zero
    correct_predictions = sum(1 for i in range(min_length) if actual_sequence[i] == predicted_sequence[i])
    accuracy = (correct_predictions / min_length) * 100  # Calculate accuracy as a percentage
    return accuracy

 
# Folder paths and files
image_folder = r"//home//jaykishor_c//ml//color_window_double1//color_window_double1"
label_file = r"//home//jaykishor_c//ml//gt_WIndow .xlsx"
output_file = r"//home//jaykishor_c//ml//Output2//BLEU_Scores_Accuracy_actual_and_predicated_char.xlsx"
 
# Load the labels DataFrame
label_df = pd.read_excel(label_file)
 
# List all image files in the folder
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]
 
# Initialize a results list and variable to track total accuracy
results = []
total_accuracy = 0
num_images = 0
 
# Process each image
for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    # Load the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        print(f"Error loading image: {image_name}")
        continue
 
    # Extract the actual sequence from the DataFrame
    actual_sequence = get_actual_sequence_from_df(image_name, image, label_df)
    if actual_sequence is None:
        print(f"No label found for image: {image_name}")
        continue
 
    # Predict the sequence using the HMM models
    predicted_sequence = predict_line_sequence(image)
    # Calculate BLEU score
    # bleu_score = calculate_bleu_score(actual_sequence, predicted_sequence)
    # Calculate accuracy
    accuracy = calculate_accuracy(actual_sequence, predicted_sequence)
    print(f'{image_name},   |   {accuracy:.2f}%')
    # Add the accuracy to the total
    total_accuracy += accuracy
    num_images += 1
    # Append results
    results.append({
        "Image Name": image_name,
        "Actual Sequence": ''.join(actual_sequence),
        "Predicted Sequence": predicted_sequence,
        # "BLEU Score": bleu_score,
        "Accuracy": accuracy
    })
 
# Calculate the average accuracy
average_accuracy = total_accuracy / num_images if num_images > 0 else 0
 
# Save the results to an Excel sheet
results_df = pd.DataFrame(results)
results_df.to_excel(output_file, index=False)
 
# Print average accuracy
print(f"Average Accuracy: {average_accuracy:.2f}%")
print(f"Results saved to {output_file}")

MaI14_007_4.jpg_window_16.jpg,   |   64.29%
MaI14_007_9.jpg_window_31.jpg,   |   0.00%
MaI14_051_07.jpg_window_16.jpg,   |   0.00%
MaI12_Page102_line_3.jpg_window_32.jpg,   |   8.33%
MaI849_041_line_5.jpg_window_51.jpg,   |   5.00%
MaI849_038_line_5.jpg_window_23.jpg,   |   4.17%
MaI14_051_04.jpg_window_40.jpg,   |   92.86%
MaI12_Page101_line_1.jpg_window_39.jpg,   |   6.67%
MaI14_007_6.jpg_window_16.jpg,   |   82.14%
MaI12_Page101_line_5.jpg_window_40.jpg,   |   10.71%
MaI849_039_line_8.jpg_window_32.jpg,   |   12.50%
MaI849_041_line_7.jpg_window_50.jpg,   |   0.00%
MaI849_041_line_2.jpg_window_27.jpg,   |   10.71%
MaI849_038_line_3.jpg_window_7.jpg,   |   32.14%
MaI12_Page101_line_5.jpg_window_49.jpg,   |   60.00%
MaI12_Page101_line_4.jpg_window_37.jpg,   |   0.00%
maI16_01_4.jpg_window_39.jpg,   |   12.50%
maI16_01_7.jpg_window_14.jpg,   |   28.57%
maI16_01_5.jpg_window_25.jpg,   |   14.29%
MaI14_051_03.jpg_window_48.jpg,   |   20.83%
MaI12_Page102_line_5.jpg_window_22.jpg,   |   12

# BLUE score for each character

In [46]:
from nltk.translate.bleu_score import sentence_bleu

characters_to_remove = ['ണ',' ൾ','ു','ഥ','ൃ','ബ','ൂഃ','ഃ','ൻ']
 
# Calculate BLEU score for predictions
for char, model in character_hmms.items():
    try:
        sequences = combined_features_glcm_and_dct[char]
        # Remove empty sequences
        sequences = [seq for seq in sequences if len(seq) > 0]
        # Check if we have any sequences left
        if len(sequences) >= 15:
            X = np.vstack(sequences)
            lengths = [len(seq) for seq in sequences]
            # Check for valid transition matrix
            row_sums = model.transmat_.sum(axis=1)
            if not np.allclose(row_sums, 1):
                print(f"Problem with transition matrix for character '{char}': row sums = {row_sums}")
                characters_to_remove.append(char)
                continue  # Skip this model and move to the next one
            # Get predicted states
            predicted_states = model.predict(X)
            mapped_predictions = [state_to_char[state] for state in predicted_states]
            # Calculate BLEU score
            bleu_score = sentence_bleu([ground_truth], mapped_predictions)
            print(f"BLEU Score for character '{char}': {bleu_score}")
    except ValueError as e:
        print(f"Error with character '{char}': {e}")

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

BLEU Score for character 'െ': 2.8389115490445386e-232
BLEU Score for character 'ത': 3.395489862661054e-232
BLEU Score for character 'അ': 3.905171559289336e-232
BLEU Score for character '്': 3.102670747246022e-232
BLEU Score for character 'ാ': 2.8074745719404793e-232
BLEU Score for character 'ി': 3.1704930077296505e-232
BLEU Score for character 'പ': 3.717992382547305e-232
BLEU Score for character 'റ': 3.450944764551663e-232
BLEU Score for character 'ര': 3.1541659018010075e-232
BLEU Score for character 'ള': 5.0812780049985006e-232


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

BLEU Score for character 'ക': 3.1750531739989355e-232
BLEU Score for character 'ഷ': 5.2822531843433194e-232
BLEU Score for character 'മ': 3.637495942076716e-232
BLEU Score for character 'ആ': 5.4761369592005975e-232
BLEU Score for character 'ഠ': 5.132511014697576e-232
BLEU Score for character 'ൽ': 5.6383456886193345e-232
BLEU Score for character 'ർ': 5.6064713439997805e-232
BLEU Score for character 'ഇ': 5.882492291217442e-232
BLEU Score for character 'ങ': 6.573479617511883e-232
BLEU Score for character 'ന': 3.215678839099107e-232
BLEU Score for character 'ജ': 4.479906922372853e-232
BLEU Score for character 'ട': 3.5871414554869394e-232
BLEU Score for character 'വ': 3.768861368173813e-232
BLEU Score for character 'ഹ': 5.377379257150705e-232
BLEU Score for character 'ം': 4.25444340014233e-232
BLEU Score for character 'യ': 3.668116985093867e-232
BLEU Score for character '"': 2.7919579467428226e-232
BLEU Score for character 'ധ': 5.132511014697576e-232
BLEU Score for character 'ച': 3.93385968

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Remove characters if they exist in the dictionary

In [47]:
for char in characters_to_remove:
    print (char)
    if char in character_hmms:
        del character_hmms[char]
        print(f"Deleted HMM model for character: {char}")
    else:
        print(f"Character {char} not found in character_hmms.")

ണ
Character ണ not found in character_hmms.
 ൾ
Character  ൾ not found in character_hmms.
ു
Character ു not found in character_hmms.
ഥ
Character ഥ not found in character_hmms.
ൃ
Character ൃ not found in character_hmms.
ബ
Character ബ not found in character_hmms.
ൂഃ
Character ൂഃ not found in character_hmms.
ഃ
Character ഃ not found in character_hmms.
ൻ
Character ൻ not found in character_hmms.


# Calculate BLEU score for predictions

In [48]:
# Calculate BLEU score for predictions (this is more useful for sequence generation tasks)
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have any sequences left
    if len(sequences) >=15:
       
        X = np.vstack(sequences)
    
    lengths = [len(seq) for seq in sequences]
    
    # Get predicted states
    predicted_states = model.predict(X)
    mapped_predictions = [state_to_char[state] for state in predicted_states]

# Calculate BLEU score
bleu_score = sentence_bleu([ground_truth], mapped_predictions)
print(f"BLEU Score: {bleu_score}")

BLEU Score: 7.817228731469945e-232


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

# Save the BLUR score to an Excel file

In [49]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu
import numpy as np

# List to store results
results = []

# Loop through each character and its model
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    
    # Remove empty sequences
    sequences = [seq for seq in sequences if len(seq) > 0]
    
    # Check if we have enough sequences to proceed
    if len(sequences) >= 15:
        X = np.vstack(sequences)

        lengths = [len(seq) for seq in sequences]
        
        # Get predicted states and map to characters
        predicted_states = model.predict(X)
        mapped_predictions = [state_to_char[state] for state in predicted_states]
        
        # Calculate BLEU score
        bleu_score = sentence_bleu([ground_truth], mapped_predictions)
        
        # Store the results
        results.append({
            'Character': char,
            'Mapped Predictions': ''.join(mapped_predictions),
            'BLEU Score': bleu_score
        })

# Create a DataFrame to store results
results_df = pd.DataFrame(results)


# Save the results to an Excel file
output_path = r'//home//jaykishor_c//ml//Output2//mapped_predictions_bleu_scores.xlsx'
results_df.to_excel(output_path, index=False)
print(f"Results saved to {output_path}")

/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

Results saved to //home//jaykishor_c//ml//Output2//mapped_predictions_bleu_scores.xlsx


/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/jaykishor_c/ml/JaykishorEnv/lib/python3.10/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider 

In [50]:
results_df

,Character,Mapped Predictions,BLEU Score
0,െ,"അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ""അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅ...",2.838912e-232
1,ത,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""...",3.395490e-232
2,അ,"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""അഅഅഅഅഅഅഅ""""""""...",3.905172e-232
3,്,"ഃഃഃഃഃഃഃഃഃഃ""""""""ഃഃഃഃഃഃ""""""""""""""""ഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃ...",3.102671e-232
4,ാ,ംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംം...,2.807475e-232
5,ി,"ഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃ""""""""ഃഃഃഃഃഃഃഃഃ...",3.170493e-232
6,പ,അഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅഅംം...,3.717992e-232
7,റ,ംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംംം...,3.450945e-232
8,ര,ംംംഃഃഃഃഃഃഃഃഃംംംംംംംംംംംംംംംംഃഃഃഃഃഃംംംംംംംംംംംം...,3.154166e-232
9,ള,ംംംംംംംംംംംംംംംംംംംംംംഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃഃ...,5.081278e-232


# Calculate Probability

**1. Log Probability**

In [ ]:
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        print(f"Log Probability for character '{char}': {log_probability}")


'for char, model in character_hmms.items():\n    sequences = combined_features_glcm_and_dct[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        print(f"Log Probability for character \'{char}\': {log_probability}")\n'

**2. Model Probability**

In [ ]:
for char, model in character_hmms.items():
    sequences = combined_features_glcm_and_dct[char]
    sequences = [seq for seq in sequences if len(seq) > 0]

    if len(sequences) >= 15:
        X = np.vstack(sequences)
        lengths = [len(seq) for seq in sequences]
        log_probability = model.score(X, lengths)
        model_probability = np.exp(log_probability)  # Convert to regular probability
        print(f"Model Probability for character '{char}': {model_probability}")


'for char, model in character_hmms.items():\n    sequences = combined_features_glcm_and_dct[char]\n    sequences = [seq for seq in sequences if len(seq) > 0]\n\n    if len(sequences) >= 15:\n        X = np.vstack(sequences)\n        lengths = [len(seq) for seq in sequences]\n        log_probability = model.score(X, lengths)\n        model_probability = np.exp(log_probability)  # Convert to regular probability\n        print(f"Model Probability for character \'{char}\': {model_probability}")\n'

**3. Transition Matrix Validation**

In [ ]:
for char, model in character_hmms.items():
    row_sums = model.transmat_.sum(axis=1)
    if not np.allclose(row_sums, 1):
        print(f"Transition matrix issue for '{char}': {row_sums}")
    else:
        print(f"Transition matrix for '{char}' is valid.")


'for char, model in character_hmms.items():\n    row_sums = model.transmat_.sum(axis=1)\n    if not np.allclose(row_sums, 1):\n        print(f"Transition matrix issue for \'{char}\': {row_sums}")\n    else:\n        print(f"Transition matrix for \'{char}\' is valid.")\n'

# Actual and Prediction of character

In [56]:
# Dictionary to store actual vs predicted characters
results = {'Actual': [], 'Predicted': []}
 
# Process each test character and its sequences
for actual_char, test_sequences in combined_features_glcm_and_dct.items():
    for test_sequence in test_sequences:
        test_sequence = np.array(test_sequence)
        # Dictionary to store scores for each character model
        scores = {}
        # Score the test sequence against each character's HMM model
        for char, model in character_hmms.items():
            try:
                scores[char] = model.score(test_sequence)
            except:
                scores[char] = -np.inf  # If model can't score, assign a low score
        # Predict the character with the highest score
        predicted_char = max(scores, key=scores.get)
        # Store the result
        results['Actual'].append(actual_char)
        results['Predicted'].append(predicted_char)
 
# Save the results to an Excel file
output_path = r'/home/jaykishor_c/ml/Output2//actual_vs_predicted.xlsx'
results_df = pd.DataFrame(results)
results_df.to_excel(output_path, index=False)


print(f"Results saved to {output_path}")

Results saved to /home/jaykishor_c/ml/Output2//actual_vs_predicted.xlsx
